# Thermostat demo

In [1]:
import warnings
# Suppress warnings
warnings.filterwarnings('ignore')

In [2]:
!pip3 install --upgrade pip
!pip3 install cmake
!pip3 install cython
!pip3 install numpy
!pip3 install torch
!pip3 install datasets
!pip3 install spacy
!pip3 install sentencepiece
!pip3 install transformers
!pip3 install overrides
!pip3 install jsonnet

Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable


Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable


In [3]:
import sys
# Include root directory in module path
sys.path.append('src')

from datasets import load_dataset
from pprint import pprint

import thermostat  # Accompanying library

# Load dataset

This will use the dataset script ("thermostat.py") in the "thermostat" directory.
In this example, we use the test set of the "imdb-bert-lgxa" configuration.
This refers to Layer Gradient x Activation (LGxA) explanations of the predictions by a BERT model that has been fine-tuned on the IMDb (train) dataset and evaluated on the IMDb test dataset.
In other words, we load the 25k test examples from the IMDb test plus the BERT predictions and the feature attributions from a Layer Gradient x Activation explainer.

In [18]:
data = load_dataset("thermostat", "imdb-bert-lig", split="test")

Dataset thermostat downloaded and prepared to /home/nfel/.cache/huggingface/datasets/thermostat/imdb-bert-lig/1.0.0/82ada9d63d3c6b421a4ade89adc656b856fe9924abbc5cc94f20d472f5c71e99. Subsequent calls will reuse this data.


Now let's inspect a single instance of the loaded dataset.
Here, we will stick to the functionality that the datasets library already supplies us with.
For readability purposes, we will not print the whole content of that instance.
Instead, we're showing only the first few entries of the attributions and the input_ids, respectively.

In [19]:
instance = data[0]

print(f'Keys: {instance.keys()}\n')
print(f'Index: {instance["idx"]}')
print(f'Input IDs (first 15): {instance["input_ids"][:15]}')
print(f'Attributions (first 4): {instance["attributions"][:4]}')
print(f'True label: {instance["label"]}')
print(f'Predictions (logits): {instance["predictions"]}')

Keys: dict_keys(['attributions', 'idx', 'input_ids', 'label', 'predictions'])

Index: 0
Input IDs (first 15): [101, 1996, 3553, 2016, 4152, 1012, 1012, 1012, 2003, 2019, 2396, 3993, 4516, 7149, 2007]
Attributions (first 4): [0.0, -0.009474949911236763, 0.20291052758693695, -0.04237872734665871]
True label: 1
Predictions (logits): [-4.403196811676025, 4.449326992034912]


# Visualize data
Can we make this a bit more readable?  
Of course! First, let's select a small subset using the datasets ".select" function:

In [20]:
data_head = data.select(range(20))

Next, we can import the "Thermopack" class from our accompanying library. It inherits all properties from a Hugging Face Dataset, but also instantiates the tokenizer of the downstream model and automatically decodes the Input IDs to words.

In [31]:
tp = thermostat.Thermopack(data_head)
print(tp)

IMDb dataset, BERT model, Layer Integrated Gradients explanations
Explainer: LayerIntegratedGradients
Model: textattack/bert-base-uncased-imdb
Dataset: imdb



In [32]:
pprint({k: v for k, v in vars(tp).items() if not k.startswith('_')})

{'dataset': Dataset({
    features: ['attributions', 'idx', 'input_ids', 'label', 'predictions'],
    num_rows: 20
}),
 'dataset_name': 'imdb',
 'explainer_name': 'LayerIntegratedGradients',
 'label_names': ['neg', 'pos'],
 'model_name': 'textattack/bert-base-uncased-imdb',
 'tokenizer': PreTrainedTokenizerFast(name_or_path='textattack/bert-base-uncased-imdb', vocab_size=30522, model_max_len=512, is_fast=True, padding_side='right', special_tokens={'unk_token': '[UNK]', 'sep_token': '[SEP]', 'pad_token': '[PAD]', 'cls_token': '[CLS]', 'mask_token': '[MASK]'}),
 'units': [<thermostat.data.dataset_utils.Thermounit object at 0x7fd1f2530d68>,
           <thermostat.data.dataset_utils.Thermounit object at 0x7fd119d1bd30>]}


In [33]:
tu_example = tp[9]
pprint({k: v for k, v in vars(tu_example).items() if not k.startswith('_') and k not in ['heatmap', 'instance', 'tokens']})

{'dataset_name': 'imdb',
 'explainer_name': 'LayerIntegratedGradients',
 'index': 9,
 'model_name': 'textattack/bert-base-uncased-imdb',
 'predicted_label': {'index': 0, 'name': 'neg'},
 'text': "i don't mind the odd artsy film. but when they are larded with "
         "arcane symbolism and murky dialogue and when it's obvious they were "
         "done for the filmmaker's ego rather than the viewer's benefit, i get "
         "upset. i'm not a stupid person yet i simply didn't understand what "
         'this film was trying to say. or do. film is a magnificent form of '
         'human communication. why do some filmmakers use it instead for '
         'obfuscation?',
 'tokenizer': PreTrainedTokenizerFast(name_or_path='textattack/bert-base-uncased-imdb', vocab_size=30522, model_max_len=512, is_fast=True, padding_side='right', special_tokens={'unk_token': '[UNK]', 'sep_token': '[SEP]', 'pad_token': '[PAD]', 'cls_token': '[CLS]', 'mask_token': '[MASK]'}),
 'true_label': {'index': 0, 'n

In [34]:
tu_example.set_heatmap(fuse_subwords_strategy='salient')
heatmap = tu_example.render(jupyter=True)
heatmap

# Aggregate data
Let us first compare the heatmaps of two different models on the same dataset+explainer configuration, MNLI + Occlusion.

In [12]:
xlnet = load_dataset("thermostat", "mnli-electra-occ", split="test")
xlnet_head = xlnet.select(range(20))
tp_xlnet = thermostat.Thermopack(xlnet_head)

bert = load_dataset("thermostat", "mnli-bert-occ", split="test")
bert_head = bert.select(range(20))
tp_bert = thermostat.Thermopack(bert_head)

Reusing dataset thermostat (/home/nfel/.cache/huggingface/datasets/thermostat/mnli-electra-occ/1.0.0/82ada9d63d3c6b421a4ade89adc656b856fe9924abbc5cc94f20d472f5c71e99)


Dataset thermostat downloaded and prepared to /home/nfel/.cache/huggingface/datasets/thermostat/mnli-bert-occ/1.0.0/82ada9d63d3c6b421a4ade89adc656b856fe9924abbc5cc94f20d472f5c71e99. Subsequent calls will reuse this data.


### Rank correlation
We can also investigate how well two explainers on the same dataset+model combination correlate regarding their attribution scores.